In [ ]:
from os import chdir, getcwd, path, makedirs
from numpy import nan
from time import time
from logging import info, error
import subprocess
from yaml import YAMLError, safe_load, dump
import datetime
from re import sub
from string import punctuation, ascii_letters
from random import random, choice, randint, uniform
from psutil import virtual_memory
import gc
from graphviz import Source, render
from objgraph import show_refs
from pandas import DataFrame, read_csv
from dask import dataframe as dd
import ray
from modin import pandas as mpd
import grpc

In [4]:
cwd = getcwd()
print(cwd)
chdir('C:/Users/andre/Repositories/Professional.Portfolio/Sample6_Data.Validation.Pipeline')
print(cwd)


C:\Users\andre\Repositories\Professional.Portfolio\Sample6_Data.Validation.Pipeline
C:\Users\andre\Repositories\Professional.Portfolio\Sample6_Data.Validation.Pipeline


## Helper Functions

In [5]:
# function to keep track of df objects in memory
def dfs():
    """
    List all Pandas DataFrame objects currently in memory.

    This function lists all Pandas DataFrame objects present in the global namespace.

    Returns:
        list: A list of DataFrame objects.
    """
    dataframes = [var for var in globals() if isinstance(globals()[var], DataFrame)]
    print(dataframes)

In [4]:
dfs()

[]


In [1]:
from psutil import virtual_memory
def monitor_ram():
    """
    Get information about the system's RAM (Random Access Memory) usage.

    Returns:
        psutil._common.svmem: A named tuple representing RAM usage statistics.
    """
    memory_usage = virtual_memory()
    return memory_usage

In [6]:
monitor_ram()

svmem(total=34033319936, available=25061523456, percent=26.4, used=8971796480, free=25061523456)

In [ ]:
def generate_ram_graph(working_dir, dot_name, objects):
    """
    Generate a reference graph of Python objects and save it as a PNG image.

    Parameters:
        working_dir (str): The directory where the DOT file and PNG image will be saved.
        dot_name (str): The base name for the DOT and PNG files (without extensions).
        objects: The Python objects for which references will be graphed.

    Returns:
        graphviz.files.Source: A Graphviz Source object representing the rendered reference graph.
    """
    # Specify the full file path where the DOT file should be saved
    dot_file_path = f'{working_dir}/{dot_name}.dot'
    
    # Generate the object references graph and save it as a DOT file
    show_refs(objects, filename=dot_file_path)
    
    # Specify the full file path where the PNG image should be saved
    png_file_path = f'{working_dir}/{dot_name}.png'
    
    # Use graphviz to render the DOT file as a PNG image
    graph = Source.from_file(dot_file_path, format='png')
    
    return graph


In [ ]:
generate_ram_graph(cwd, 'graph', monitor_ram())

## Create Mock Dataset with Special Characters, Strings, Integers, & Missing Values

In [14]:
def generate_random_value():
    """
    Generate a random value with various data types and potential missing values.

    Returns:
        str, int, float, or None: A randomly generated value, which can be a string, integer, float,
        or None (representing a missing value).

    Description:
        This function generates random values with different data types and the possibility of missing values.
        - 10% chance of returning a missing value (None).
        - 20% chance of returning a string consisting of 5 random special characters.
        - 50% chance of returning a string consisting of 10 random alphanumeric characters.
        - 20% chance of returning either a random integer between 1 and 1000 (inclusive) or
          a random float between 0.1 and 1000.0 (inclusive).

    Example:
        Possible outputs:
        - 'ABc!@#' (string with special characters)
        - 123 (integer)
        - 456.789 (float)
        - None (missing value)
    """
    
    # 10% chance of missing value
    if random() < 0.1:  
        return nan
    # 20% chance of special character
    elif random() < 0.2:  
        return ''.join(choice(punctuation) for _ in range(5))
    # 50% chance of string
    elif random() < 0.5:  
        return ''.join(choice(ascii_letters) for _ in range(10))
     # 20% chance of number (integer or float)
    else: 
        return choice([randint(1, 1000), uniform(0.1, 1000.0)])

In [27]:
def generate_and_save_large_dataframe(file_type, postfix, delimiter=None):
    """
    Generate a large Pandas DataFrame with random data and save it to a CSV file when it exceeds 2GB in size.

    Description:
        This function generates random data and creates a Pandas DataFrame. It keeps adding rows to the DataFrame
        until its size exceeds 2GB. Once the size limit is reached, the DataFrame is saved to a CSV file
        with a filename indicating the size of the dataset in millions of rows (e.g., 'mock_dataset_6M.csv').

    Note:
        The function uses the `generate_random_value` function to create random data.

    Returns:
        None
    """
    start_time0 = time()
    data = {}
    num_columns = 25
    num_rows = 6000000

    while True:
        # Generate dict substructure for a Pandas dataframe
        for i in range(num_columns):
            column_name = f'column_{i+1}'
            data[column_name] = [generate_random_value() for _ in range(num_rows)]

        # Create a Pandas DataFrame
        df = DataFrame(data)
        df_size_bytes = df.memory_usage(index=True).sum()

        # Check size of dataframe: if > than 2GB, save the file
        if df_size_bytes > 2 * 1073741824: 
            filename = f'mock_dataset_{num_rows//1000000}M{postfix}.{file_type}'
            if file_type == 'parquet':
                df.to_parquet(filename, index=False, engine='fastparquet')
            elif file_type == 'csv':
                df.to_csv(filename, index=False, sep=delimiter)
            elif file_type == 'tsv':
                df.to_csv(filename, index=False, sep=delimiter)
            print(f"Saved DataFrame to {filename}")
            break  
        # Else, add another 2000000 rows to the df
        else:
            del df 
            num_rows += 2000000

    end_time0 = time() - start_time0
    print(end_time0)


## Decide Library to Use to Speed Up Validation

In [76]:
## > Pandas

def load_csv_with_pandas(file_path):
    """
    Load a CSV file using Pandas and measure the loading time.

    Parameters:
        file_path (str): The path to the CSV file.

    Returns:
        float: The time taken to load the CSV file using Pandas (in seconds).
    """
    start_time1 = time()
    df = read_csv(file_path)
    end_time1 = time() - start_time1
    print(end_time1)
    return df

In [7]:
## > Dask

def load_csv_with_dask(file_path):
    """
    Load a CSV file using Dask and measure the loading time.

    Parameters:
        file_path (str): The path to the CSV file.

    Returns:
        float: The time taken to load the CSV file using Dask (in seconds).
    """
    start_time2 = time()
    ddf = dd.read_csv(file_path)
    end_time2 = time() - start_time2
    print(end_time2)
    return ddf


In [78]:
## > Ray

ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})


2024-01-18 12:27:52,074	INFO worker.py:1724 -- Started a local Ray instance.


Python version:,3.11.7
Ray version:,2.9.0


In [79]:
## > Modin

def load_csv_with_modin(file_path):
    """
    Load a CSV file using Modin and measure the loading time.

    Parameters:
        file_path (str): The path to the CSV file.

    Returns:
        float: The time taken to load the CSV file using Modin (in seconds).
    """
    start_time3 = time()
    mdf = mpd.read_csv(file_path)
    end_time3 = time() - start_time3
    print(end_time3)
    return mdf


# Automated Data Validation Pipeline: Using Dask

## Utility functions

In [8]:

def read_config_file(filepath):
    
    """
    Read and parse a YAML configuration file.

    Parameters:
        filepath (str): The path to the YAML configuration file.

    Returns:
        dict: A dictionary containing the parsed configuration data.
    """
    
    # Open specified filepath
    with open(filepath, 'r') as datacreek:
        # Try-Catch for YAMLError
        try:
            return safe_load(datacreek)
        except YAMLError as exc:
            # Logging library error sent to 'stdout'
            error(exc)
            

def replacer(string, char):
    """
    Replace consecutive instances of a character in a string with a single instance.

    Parameters:
        string (str): The input string.
        char (str): The character to be replaced.

    Returns:
        str: The input string with consecutive instances of the character replaced.
    """
    
    # Regex pattern for 2 or more instances
    pattern = char + '{2,}'
    
    # Sub function to replace specified char
    string = sub(pattern, char, string)
    
    # Returns the string with replaced char
    return string



def col_header_val(df, table_config):
    """
    Validate and standardize column names in a DataFrame based on a table configuration.

    Parameters:
        df (dd.DataFrame): The Dask DataFrame to be validated.
        table_config (dict): The table configuration dictionary.

    Returns:
        bool: True if validation passes, False otherwise.
    """
    
    # Convert all strings to lowercase
    df.columns = df.columns.str.lower()
 
    # Replace all whitespce at the start of column names
    df.columns = df.columns.str.replace('[^\w]', '_', regex=True)
    
    # Removes underscores from beginning & end of column names
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
   
    # Replaces 2 or more consecutive underscores with a single underscore
    df.columns = list(map(lambda x: replacer(x, '_'), list(df.columns)))
    
    # Converts expected col_names for 'table_config' to ensure case insensitivity during comaparison
    expected_col = list(map(lambda x: x.lower(), table_config['columns']))
   
  
    # Converts all column names in df once again. Ensures case insensitivity when comparing with expected col_names
    df.columns = list(map(lambda x: x.lower(), list(df.columns)))
    
    # Sort the DataFrame by multiple columns
    df = df[table_config['columns']]

    if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
        print('column namd and column length validation passed')
        return True

    # If the above is false, then we check what the differences are between df.col and exp_col and print them
    else:
        print('column name and column length validation failed')
        # Uses set operations for taking the difference between df.col and exp_col
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print('The following YAML columns are not in the YAML file', mismatched_columns_file )
        # Uses set operations to check diff between exp_col and df_col
        missing_yaml_file = list(set(expected_col).difference(df.columns))
        print('The following YAML columns are not in the uploaded file', missing_yaml_file)
        # log results
        info(f'df columns: {df.columns}')
        info(f'expected columns: {expected_col}')
        return False


def generate_yaml_config(file_type, file_name, table_name, columns, in_del, out_del):
    """
    Generate a YAML configuration file for data processing and save it.

    Parameters:
        file_type (str): The type of the data file (e.g., 'csv', 'parquet').
        file_name (str): The name of the data file.
        table_name (str): The name of the data processing table.
        columns (list): A list of column names.
        in_del (str): The inbound delimiter character.
        out_del (str): The outbound delimiter character.

    Returns:
        tuple: A tuple containing the generated file_name, file_type, and table_name.
    """
    # Create a configuration dictionary
    config_data = {
        'file_name': file_name,
        'file_type': file_type,
        'table_name': table_name,
        'inbound_delimiter': in_del,
        'outbound_delimiter': out_del,
        'skip_leading_rows': 1,
        'columns': columns,
    }

    # Convert the dictionary to YAML format
    yaml_config = dump(config_data, default_flow_style=False)

    # Save the YAML configuration to a file
    with open(f'{table_name}_config.yaml', 'w') as yaml_file:
        yaml_file.write(yaml_config)
        
    return file_name, file_type, table_name




def read_file(file_type, source_file):
    """
    Read data from a file into a Dask DataFrame based on the file type.

    Parameters:
        file_type (str): The type of the data file (e.g., 'csv', 'excel', 'parquet').
        source_file (str): The path to the source data file.

    Returns:
        dd.DataFrame: A Dask DataFrame containing the data from the file.
    """
    if file_type == 'csv':
        return dd.read_csv(source_file)
    elif file_type == 'tsv':
        return dd.read_csv(source_file)
    elif file_type == 'parquet':
        return dd.read_parquet(source_file)
    # Add more file types as needed...
    
    


def save_file(df, file_type, target_path):
    """
    Save a Dask DataFrame to a file with the specified file type.

    Parameters:
        df (dd.DataFrame): The Dask DataFrame to be saved.
        file_type (str): The file type (e.g., 'csv', 'parquet', 'excel').
        target_path (str): The path where the file should be saved.

    Returns:
        bool: True if the file was saved successfully or already exists, False otherwise.
    """
    try:
        if file_type == 'csv':
            if path.exists(target_path):
                print(f"File already exists: {target_path}")
            else:
                df.to_csv(target_path, single_file=True)
        elif file_type == 'tsv':
            if path.exists(target_path):
                print(f"File already exists: {target_path}")
            else:
                df.to_parquet(target_path)
        elif file_type == 'parquet':
            if path.exists(target_path):
                print(f"File already exists: {target_path}")
            else:
                df.to_excel(target_path, index=False)
        # Add more file types as needed
        else:
            print(f"Unsupported file type: {file_type}")
            return False

        print(f"File saved successfully: {target_path}")
        return True
    except Exception as e:
        print(f"Error saving file: {e}")
        return False



## Main Script

In [28]:
# Create 2.6GB dataset. Time to create: ~20min
# generate_and_save_large_dataframe(file_type='csv', '0')
# generate_and_save_large_dataframe(file_type='tsv', postfix='1', delimiter='\t')
generate_and_save_large_dataframe(file_type='parquet', postfix='2')

ValueError: Can't infer object conversion type: 0           OfchqwZFwa
1                  358
2           iZZVpCYXCD
3                !.>&^
4           DUttTIqkUH
               ...    
11999995    imdOjoobPm
11999996    hrcgbekBlX
11999997    YsUGDUvFtm
11999998    HPhYyEzDOE
11999999            32
Name: column_1, Length: 12000000, dtype: object

In [81]:
# Load 2.6Gb dataset with pandas and evaluate time taken
load_csv_with_pandas('./mock_dataset_12M0.csv')

65.25137448310852


,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,...,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25
0,451,olCKjEJRTb,"\~#.""",jPvYRujZDS,bGcouhmofj,KiQxDitrFY,jSYgRKeyQq,jpUUVyLvwb,WWCUiYiELx,^>;*~,...,NaN,WgHLerArpN,OwfMPIOgUt,NaN,581.1607915309216,DPDQfiAQar,JRlSvivkjU,345,790.7288539682944,mscUHDaDZM
1,280,kbFmugeeVo,894,GPyUUbtzKu,627,zadPktaxTR,251,NaN,319.6901201341251,XHFVYWSEqw,...,344.76672767313215,NaN,NaN,jztCHUoEyr,NaN,767.2678052402874,jiBxbuRgmt,750,SJvgCBBjpD,!^>>`
2,FpctnqCfDp,NaN,MtFqmZwuoC,HAELOxyWoP,"((,^,",442,llSdzBbPqr,"""'(>/",@/|!},465,...,NaN,687.2438312191324,\)_]],tQBYtGVamC,"^$]""(",WyUZuPaYAN,ydvGwhNbFV,LyWtYFpQIG,",@_"";",NaN
3,234,3,310,%$#:|,jhumAGRbrz,[[;*#,282.0077388587002,NaN,onxyHYARXM,-%`@#,...,395,jxfRdeOXJv,NaN,";_/;,","-$]},",;+*=-,371,629.1299360393232,&\=(~,767.664953908018
4,tuLswYaBkI,938,tOSCtoNhvJ,sgZqiedejD,941,sGzxUgXwZk,'_|<[,29,",&${]",-./??,...,142,TeUinHJHgB,uYyjxznqQZ,21.75444810677572,"/~-!""",186,421,285.8714278132402,UHrCdOdrqz,156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11999995,GQxziloHLj,307,NaN,391,",`^%\",235.87822305977667,}.#;{,336.1922066718312,619,814,...,MjxastykDs,391.7840664122309,wGXEhfDgLM,837,/%?_|,SIggckQiQl,.\&+),47,|->};,NaN
11999996,419,745.4067189729501,cWaJqsqBNn,dOFXwjQYpW,685.5622420143548,PTfBmCsNsB,272.4983454541967,657.5530976063063,MvEkgtBDxC,115.85215903418788,...,660.2359883208311,{|{@-,\$>%>,qFbnvSiTep,NaN,NaN,965,PTiBshHODN,484.1495409206947,mTlfsUjmek
11999997,$}^.),BLligpLMdl,NaN,URxLDcfSeh,816.7925045401448,@:/|^,gCqbcZmnje,733,uPKMuamjOM,HlNlOLKuXT,...,qOAmoNtswJ,720,MotBbTwZap,lZJMteSqgq,%+>%!,983.7241456629364,xaeDeGapdS,mDIUkMlwtM,"-{-""}","{#"";@"
11999998,icRgSQlEHt,682.8790864608782,DxILBqPXpO,NaN,ZkamBBoPlI,|^#~~,bkgLTKwnkO,168,MCIiGGXdxM,*^/<^,...,qiTkvXBoRE,xLtnnXNFJa,536.5822285811877,VuOfgFIuLS,jjnwfEUcVz,NaN,238.05003063534798,121.57209688507182,910.902205326873,_^?>`


""


In [9]:
ddf0 = load_csv_with_dask('./mock_dataset_12M0.csv')
ddf1 = load_csv_with_dask('./mock_dataset12M1.tsv')
ddf2 = load_csv_with_dask('./mock_dataset_12M2.parquet')

display(dfs())
display(monitor_ram())

0.0242917537689209
[]


None

svmem(total=34033319936, available=25059729408, percent=26.4, used=8973590528, free=25059729408)

In [83]:
load_csv_with_modin('./mock_dataset_12M0.csv')

81.16706705093384


,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,...,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25
0,451,olCKjEJRTb,"\~#.""",jPvYRujZDS,bGcouhmofj,KiQxDitrFY,jSYgRKeyQq,jpUUVyLvwb,WWCUiYiELx,^>;*~,...,NaN,WgHLerArpN,OwfMPIOgUt,NaN,581.1607915309216,DPDQfiAQar,JRlSvivkjU,345,790.7288539682944,mscUHDaDZM
1,280,kbFmugeeVo,894,GPyUUbtzKu,627,zadPktaxTR,251,NaN,319.6901201341251,XHFVYWSEqw,...,344.76672767313215,NaN,NaN,jztCHUoEyr,NaN,767.2678052402874,jiBxbuRgmt,750,SJvgCBBjpD,!^>>`
2,FpctnqCfDp,NaN,MtFqmZwuoC,HAELOxyWoP,"((,^,",442,llSdzBbPqr,"""'(>/",@/|!},465,...,NaN,687.2438312191324,\)_]],tQBYtGVamC,"^$]""(",WyUZuPaYAN,ydvGwhNbFV,LyWtYFpQIG,",@_"";",NaN
3,234,3,310,%$#:|,jhumAGRbrz,[[;*#,282.0077388587002,NaN,onxyHYARXM,-%`@#,...,395,jxfRdeOXJv,NaN,";_/;,","-$]},",;+*=-,371,629.1299360393232,&\=(~,767.664953908018
4,tuLswYaBkI,938,tOSCtoNhvJ,sgZqiedejD,941,sGzxUgXwZk,'_|<[,29,",&${]",-./??,...,142,TeUinHJHgB,uYyjxznqQZ,21.75444810677572,"/~-!""",186,421,285.8714278132402,UHrCdOdrqz,156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11999995,GQxziloHLj,307,NaN,391,",`^%\",235.87822305977667,}.#;{,336.1922066718312,619,814,...,MjxastykDs,391.7840664122309,wGXEhfDgLM,837,/%?_|,SIggckQiQl,.\&+),47,|->};,NaN
11999996,419,745.4067189729501,cWaJqsqBNn,dOFXwjQYpW,685.5622420143548,PTfBmCsNsB,272.4983454541967,657.5530976063063,MvEkgtBDxC,115.85215903418788,...,660.2359883208311,{|{@-,\$>%>,qFbnvSiTep,NaN,NaN,965,PTiBshHODN,484.1495409206947,mTlfsUjmek
11999997,$}^.),BLligpLMdl,NaN,URxLDcfSeh,816.7925045401448,@:/|^,gCqbcZmnje,733,uPKMuamjOM,HlNlOLKuXT,...,qOAmoNtswJ,720,MotBbTwZap,lZJMteSqgq,%+>%!,983.7241456629364,xaeDeGapdS,mDIUkMlwtM,"-{-""}","{#"";@"
11999998,icRgSQlEHt,682.8790864608782,DxILBqPXpO,NaN,ZkamBBoPlI,|^#~~,bkgLTKwnkO,168,MCIiGGXdxM,*^/<^,...,qiTkvXBoRE,xLtnnXNFJa,536.5822285811877,VuOfgFIuLS,jjnwfEUcVz,NaN,238.05003063534798,121.57209688507182,910.902205326873,_^?>`


In [32]:
### > Main script


if __name__ == '__main__':
    
    
    # --- CREATING - YAML - FILES --- #
    
    # List of file configurations
    file_configurations = [
    
        {
            'file_name': 'mock_dataset_12M0',
            'file_type': 'csv',
            'table_name': 'table1',
            'in_del': ',',
            'out_del': '|',
            'columns': ddf0.columns
        },
        {
            'file_name': 'mock_dataset_12M1',
            'file_type': 'tsv',
            'table_name': 'table2',
            'in_del': '\t',
            'out_del': ',',
            'columns': ddf1.columns
        },
        {
            'file_name': 'mock_dataset_12M2',
            'file_type': 'parquet',
            'table_name': 'table2',
            'columns': ddf2.columns
        }
    
        # Add more configurations as needed ...
    ]
    
    
    # Create a loop to process each configuration
    for config in file_configurations:
        file_name, file_type, table_name = generate_yaml_config(
    
                    file_type=config['file_type'],
                    file_name=config['file_name'],
                    table_name=config['table_name'],
                    columns=config['columns'],
                    in_del=config.get('in_del', ''),
                    out_del=config.get('out_del', '')
        )
        
        

        # --- READING - IN - CONFIG FILES - AND - DATASETS --- #
    
        # Read config file
        config_data = read_config_file(file_name)
    
        # Extract file type from YAML config file
        file_type = config_data['file_type']

      
        # Construct the path to the dataset using extracted file type
        source_file = './' + config_data['file_name'] + f'.{file_type}'
    
        # Read the file based on its type and path
        ddf = read_file(file_type, source_file)
        
        

        # --- CLEANSING - OPERATIONS --- #
        
        # Double instance we want to replace
        char = ','

        # Replace all instan
        ddf = ddf.apply(lambda x: replacer(x, char))
    
        # Perform column validation
        result = col_header_val(ddf, config_data)
        


        # --- SAVING - CLEANSED - DATA - AT - SPECIFIED - PATHS --- #

        if result:
            # Define the target file path and file type
            target_file_path = f'cleansed_{file_name}.{file_type}'
    
            # Save the file using the save_file function
            save_result = save_file(ddf, file_type, target_file_path)
    
            if save_result:
                print(f'Successfully processed and saved: {file_name}')
            else:
                print(f'Error saving the file for: {file_name}')
        else:
            print(f'Validation failed for: {file_name}')

    

    

ERROR:root:expected '<document start>', but found '<scalar>'
  in "mock_dataset_12M.csv", line 17, column 1


TypeError: 'NoneType' object is not subscriptable